# EfficientNet V2B0
#### tensorflow==2.18.0

In [76]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation

In [77]:
train_dir = r"C:\Users\91909\Desktop\ML\DATA\PlantVillage-Dataset-master\PlantVillage-Dataset-master\raw\PLANT_VILLAGE_SPLIT\train"
test_dir = r"C:\Users\91909\Desktop\ML\DATA\PlantVillage-Dataset-master\PlantVillage-Dataset-master\raw\PLANT_VILLAGE_SPLIT\test"
val_dir = r"C:\Users\91909\Desktop\ML\DATA\PlantVillage-Dataset-master\PlantVillage-Dataset-master\raw\PLANT_VILLAGE_SPLIT\val"

In [78]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 4

In [79]:
def preprocess_function(image):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image

In [80]:
datagen = ImageDataGenerator(rescale=1.0/255,
                             preprocessing_function=preprocess_function)

In [81]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 32572 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.
Found 10885 images belonging to 38 classes.


In [82]:
def encoder_block(x, filters):
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    return x

def decoder_block(x, filters):
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [83]:
inputs = Input(shape=(224, 224, 3))

e1 = encoder_block(inputs, 32)
e2 = encoder_block(e1, 64)
e3 = encoder_block(e2, 128)
e4 = encoder_block(e3, 256)

b = Conv2D(512, (3, 3), padding='same')(e4)
b = BatchNormalization()(b)
b = Activation('relu')(b)

d4 = decoder_block(b, 256)
d3 = decoder_block(d4, 128)
d2 = decoder_block(d3, 64)
d1 = decoder_block(d2, 32)

outputs = Conv2D(38, (1, 1), activation='softmax')(d1)  
outputs = tf.keras.layers.GlobalAveragePooling2D()(outputs)

model = Model(inputs, outputs)

In [84]:
model.compile(tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_162 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 batch_normalization_153 (Ba  (None, 224, 224, 32)     128       
 tchNormalization)                                               
                                                                 
 activation_153 (Activation)  (None, 224, 224, 32)     0         
                                                                 
 conv2d_163 (Conv2D)         (None, 224, 224, 32)      9248      
                                                                 
 batch_normalization_154 (Ba  (None, 224, 224, 32)     128       
 tchNormalization)                                         

In [86]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

Epoch 1/50
8143/8143 [==============================] - 1354s 166ms/step - loss: 2.7811 - accuracy: 0.2500 - val_loss: 2.3782 - val_accuracy: 0.3067
Epoch 2/50
1990/8143 [======>.......................] - ETA: 14:56 - loss: 2.3325 - accuracy: 0.3525

In [ ]:
def printplot(history):
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history["accuracy"], color="red", label="train")
        plt.plot(history.history["val_accuracy"], color="green", label="validation", linestyle="dashed")
        plt.title("Graph of accuracy over epochs")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history["loss"], color="red", label="train")
        plt.plot(history.history["val_loss"], color="green", label="validation", linestyle="dashed")
        plt.title("Graph of loss over epochs")
        plt.legend()

printplot(history)

NameError: name 'history' is not defined

In [ ]:
model.save("SegNet.h5")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

681/681 ━━━━━━━━━━━━━━━━━━━━ 44s 65ms/step - accuracy: 0.2235 - loss: 2.8875
Test Accuracy: 21.33%
Test Loss: 2.9110
